In [ ]:
!pip install faiss-cpu numba pandas plotly nbformat jinja2 kaleido tqdm scikit-learn scipy matplotlib

In [ ]:
from bench import *
import plotly.graph_objects as go

In [ ]:
def speed_chart(df, title):
    subtitles = [
        f'{n}, recall@1 ~{r*100:.2f}%'
        for n, r in zip(df['names'], df['recall_at_one'])
    ]
    fig = go.Figure(
        data=[
            go.Bar(name='Add', x=subtitles, y=df['add_per_second']),
            go.Bar(name='Search', x=subtitles, y=df['search_per_second']),
        ])
    fig.update_layout(
        width=1000,
        height=1000,
        barmode='group',
        title=dict(
            text=title,
        ),
        legend=dict(
            orientation='h',
            entrywidth=70,
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1,
        ),
    )
    fig.write_image(title + '.png')
    return fig

In [ ]:
from usearch.eval import Dataset
data = Dataset.build(
    vectors='../../datasets/wiki_1M/base.1M.fbin',
    queries='../../datasets/wiki_1M/query.public.100K.fbin',
    neighbors='../../datasets/wiki_1M/groundtruth.public.100K.ibin',
)

In [ ]:
eval_default = Evaluation.for_dataset(data)
results_default = bench_speed(eval_default, jit=True)
results_default

In [ ]:
speed_chart(results_default, 'USearch vs FAISS Speed')

In [ ]:
eval_sliced = Evaluation.for_dataset(data, batch_size=256)
results_sliced = bench_speed(eval_sliced, jit=True)
results_sliced

In [ ]:
speed_chart(results_sliced, 'USearch vs FAISS Speed with 256 Batch Size')

In [ ]:
eval_clustered = Evaluation.for_dataset(data, clusters=10_000)
len(eval_clustered.tasks)

In [ ]:
results_clustered = bench_speed(eval_clustered)
results_clustered

In [ ]:
speed_chart(results_clustered, 'USearch vs FAISS Speed on Clustered Data')